In [1]:
import tensorflow as tf

with tf.device('/cpu:0'):
    a_c = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a-cpu')
    b_c = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b-cpu')
    c_c = tf.matmul(a_c, b_c, name='c-cpu')

with tf.device('/gpu:0'):
    a_g = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a-gpu')
    b_g = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b-gpu')
    c_g = tf.matmul(a_g, b_g, name='c-gpu')

with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    print (sess.run(c_c))
    print (sess.run(c_g))

print('DONE!')

[[22. 28.]
 [49. 64.]]
[[22. 28.]
 [49. 64.]]
DONE!


In [2]:
import nibabel as nib

training_img = nib.load('IXI002-Guys-0828-T1.nii.gz')
training_header = training_img.header
training_pixdim = training_img.header['pixdim'][1:4]

In [3]:
print(training_header['dim'],training_header['pixdim'])
# print(training_pixdim)

[  3 256 256 150   1   1   1   1] [-1.        0.9375    0.9375    1.199997  0.        0.        0.
  0.      ]


In [4]:
t2 = nib.load('IXI029-Guys-0829-T1.nii.gz')
t2_header = t2.header
print(t2_header['dim'],t2_header['pixdim'])

[  3 256 256 150   1   1   1   1] [-1.         0.9375     0.9375     1.1999968  0.         0.
  0.         0.       ]


In [ ]:
import scipy.ndimage
import xlrd

In [50]:

data = xlrd.open_workbook('IXI.xls','rb')
print('工作表名为：'+ data.sheet_names()[0])
table = data.sheets()[0]
nrows = table.nrows
ncols = table.ncols
print('表格行数为%d,列数为%d'%(nrows,ncols))

#输出每一行的值
for item in range(table.nrows):
    print(table.row_values(item))

#获取单元格的值
cell_A1 = table.row(0)[0].value
cell_A2 = table.cell(0,0).value
cell_A3 = table.col(0)[0].value

print(cell_A1)
print(cell_A2)
print(cell_A3)

工作表名为：Table
表格行数为620,列数为12
['IXI_ID', 'SEX_ID (1=m, 2=f)', 'HEIGHT', 'WEIGHT', 'ETHNIC_ID', 'MARITAL_ID', 'OCCUPATION_ID', 'QUALIFICATION_ID', 'DOB', 'DATE_AVAILABLE', 'STUDY_DATE', 'AGE']
[1.0, 1.0, 170.0, 80.0, 2.0, 3.0, 5.0, 2.0, '1968-02-22', 0.0, '', '']
[2.0, 2.0, 164.0, 58.0, 1.0, 4.0, 1.0, 5.0, '1970-01-30', 1.0, 38674.0, 35.800136892539356]
[12.0, 1.0, 175.0, 70.0, 1.0, 2.0, 1.0, 5.0, '1966-08-20', 1.0, 38504.0, 38.78165639972622]
[13.0, 1.0, 182.0, 70.0, 1.0, 2.0, 1.0, 5.0, '1958-09-15', 1.0, 38504.0, 46.71047227926078]
[14.0, 2.0, 163.0, 65.0, 1.0, 4.0, 1.0, 5.0, '1971-03-15', 1.0, 38512.0, 34.23682409308692]
[15.0, 1.0, 181.0, 90.0, 2.0, 1.0, 6.0, 5.0, '1981-03-11', 1.0, 38526.0, 24.28473648186174]
[16.0, 1.0, 172.0, 63.0, 1.0, 2.0, 1.0, 5.0, '1950-04-24', 1.0, 38527.0, 55.167693360711844]
[17.0, 2.0, 178.0, 72.0, 1.0, 3.0, 1.0, 5.0, '1976-05-21', 1.0, 38527.0, 29.09240246406571]
[19.0, 1.0, 180.0, 88.0, 1.0, 2.0, 1.0, 4.0, '1946-11-02', 1.0, 38533.0, 58.65845311430527]
[20

In [ ]:
table.row_values(0)[0] #id
table.row_values(0)[9] #available
table.row_values(0)[11] #age

In [1]:
# from .nii.gz to .npy
# saved in u-disk

import nibabel as nib
import pdb
import numpy as np
import scipy.ndimage
import matplotlib.pyplot as plt

def resample(image, pixdim, new_spacing=[1,1,1]):
    # Determine current pixel spacing
#     pdb.set_trace()
    spacing = pixdim

    resize_factor = spacing / new_spacing
    new_real_shape = image.shape * resize_factor
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / image.shape
#     new_spacing = spacing / real_resize_factor
    
    image = scipy.ndimage.interpolation.zoom(image, real_resize_factor, mode='nearest')
    
    return image




if __name__ == '__main__':
#     pdb.set_trace()
    nii_img = nib.load('IXI002-Guys-0828-T1.nii.gz')
    header = nii_img.header
    pixdim = header['pixdim'][1:4]
    npy_img = nii_img.get_data()
    print('original image shape: ',npy_img.shape)
#     plt.imshow(npy_img[:,:,50])
    resampled_img = resample(npy_img, pixdim, [2,2,2])
    print('resampled img shape: ',resampled_img.shape)
#     a = np.array([[[1,2],[3,4],[5,6]],[[1,2],[3,4],[5,6]]])
#     plt.imshow(a, cmap=plt.cm.gray, interpolation='nearest')
#     plt.imshow(a[:,:,0])
#     plt.figure()
#     plt.imshow(resampled_img[:,:,50])
#     resampled_img = resample(resampled_img, pixdim, [2,2,2])
    
    DESIRED_SHAPE = (130, 130, 110)
    X_margin_0 = int((DESIRED_SHAPE[0]-resampled_img.shape[0])/2)
    Y_margin_0 = int((DESIRED_SHAPE[1]-resampled_img.shape[1])/2)
    Z_margin_0 = int((DESIRED_SHAPE[2]-resampled_img.shape[2])/2)
    
    X_margin_1 = DESIRED_SHAPE[0]-resampled_img.shape[0]-X_margin_0
    Y_margin_1 = DESIRED_SHAPE[1]-resampled_img.shape[1]-Y_margin_0
    Z_margin_1 = DESIRED_SHAPE[2]-resampled_img.shape[2]-Z_margin_0
    
    npad = ((X_margin_0,X_margin_1), 
            (Y_margin_0,Y_margin_1), 
            (Z_margin_0,Z_margin_1))
    padded_img = np.pad(resampled_img, pad_width=npad, mode='constant', constant_values=0)
    print(padded_img.shape)
    
    np.save('test.npy',padded_img)

original image shape:  (256, 256, 150)
resampled img shape:  (120, 120, 90)
(130, 130, 110)


In [46]:
t = np.load('test.npy')
print(t.shape)

(130, 130, 110)


In [32]:
scipy.ndimage.interpolation.zoom?

In [11]:
a = np.array([[1,2],[3,4],[5,6]])
b = np.random.rand(2,3)

In [37]:
c = np.random.randint(10,size=(8,8))
print(c)
print(scipy.ndimage.interpolation.zoom(c,[0.5,0.5],mode='nearest'))

[[3 7 7 3 1 0 9 6]
 [6 4 0 3 7 0 6 0]
 [8 7 3 2 4 1 4 1]
 [4 0 9 1 6 6 0 8]
 [6 7 1 8 7 4 3 9]
 [4 2 0 4 5 1 0 2]
 [7 0 3 4 2 2 7 6]
 [4 8 7 6 1 8 8 6]]
[[ 3  6 -1  6]
 [ 7  4  3  3]
 [ 4  1  3  4]
 [ 4  7  5  6]]


In [3]:
import random
import numpy as np
np.random?

In [48]:
134/4 + 16.9+2.5

52.9